# Preprocessing Data to `protein_embeddings.parquet`
The following cells were used to first preprocess the train.csv file to create the mutated sequences. Then, we run the ESM model to get the embeddings and store these in a parquet file for later use.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

# get the sequence
seq = open('sequence.fasta', 'r').read()
seq = seq.split("\n")[1]

# create each mutated sequence using the info
sequences = []
for i in df['mutant']:
    ind = int(i[1:-1])
    tmp = seq[:ind] + i[-1] + seq[ind+1:]
    sequences.append(tmp)
df['Sequence'] = sequences

In [3]:
import torch
import esm

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained ESM model and move it to GPU
esm_model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()  # Example: ESM-2 model
batch_converter = alphabet.get_batch_converter()
esm_model = esm_model.to(device)
esm_model.eval()  # Set to eval mode

# Extract ESM embeddings using GPU
def extract_esm_embedding(sequence):
    batch_labels, batch_strs, batch_tokens = batch_converter([(None, sequence)])
    batch_tokens = batch_tokens.to(device)  # Move input to GPU

    with torch.no_grad():
        results = esm_model(batch_tokens, repr_layers=[33], return_contacts=False)
    
    token_representations = results["representations"][33]  # Use final layer
    sequence_embedding = token_representations.mean(dim=1).squeeze().cpu().numpy()  # Move back to CPU for NumPy
    return sequence_embedding

Using device: cuda


In [4]:
# Apply embedding extraction
df["Embedding"] = df["Sequence"].apply(lambda seq: extract_esm_embedding(seq))

df.to_parquet("protein_embeddings.parquet", engine="pyarrow")

# Training Models
We initially train a MLP, LightGBM, and XGBoost.

In [5]:
df = pd.read_parquet('protein_embeddings.parquet')

In [6]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X = np.vstack(df["Embedding"].values)
y = df["DMS_score"].values

# Convert to PyTorch tensors and move to GPU
X_tensor = torch.tensor(X, dtype=torch.float32, device=device)
y_tensor = torch.tensor(y, dtype=torch.float32, device=device).view(-1, 1)  # Reshape for MLP

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)


## MLP

In [46]:
from scipy.stats import spearmanr

# Define a simple MLP regression model using GPU
class MLPRegressor(nn.Module):
    def __init__(self, input_dim):
        super(MLPRegressor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # Single output value
        )

    def forward(self, x):
        return self.model(x)

# Initialize model, loss function, and optimizer
model = MLPRegressor(input_dim=X.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 2000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    loss = criterion(predictions, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Evaluate on test set
model.eval()
with torch.no_grad():
    test_predictions = model(X_test)
    test_loss = criterion(test_predictions, y_test)
print(f"Test MSE: {test_loss.item():.4f}")

spearman_corr_xgb, _ = spearmanr(y_test.cpu().numpy().flatten(), test_predictions.cpu().numpy().flatten())
print(f"MLP Spearman Correlation: {spearman_corr_xgb:.4f}")

Epoch 100/2000, Loss: 0.0412
Epoch 200/2000, Loss: 0.0399
Epoch 300/2000, Loss: 0.0397
Epoch 400/2000, Loss: 0.0397
Epoch 500/2000, Loss: 0.0382
Epoch 600/2000, Loss: 0.0369
Epoch 700/2000, Loss: 0.0365
Epoch 800/2000, Loss: 0.0369
Epoch 900/2000, Loss: 0.0380
Epoch 1000/2000, Loss: 0.0385
Epoch 1100/2000, Loss: 0.0359
Epoch 1200/2000, Loss: 0.0352
Epoch 1300/2000, Loss: 0.0348
Epoch 1400/2000, Loss: 0.0398
Epoch 1500/2000, Loss: 0.0391
Epoch 1600/2000, Loss: 0.0412
Epoch 1700/2000, Loss: 0.0412
Epoch 1800/2000, Loss: 0.0409
Epoch 1900/2000, Loss: 0.0393
Epoch 2000/2000, Loss: 0.0381
Test MSE: 0.0483
MLP Spearman Correlation: 0.3111


## XGBoost

In [47]:
import xgboost as xgb
from scipy.stats import spearmanr

# Convert to DMatrix (XGBoost's optimized data structure)
dtrain = xgb.DMatrix(X_train.cpu().numpy(), label=y_train.cpu().numpy().flatten())
dtest = xgb.DMatrix(X_test.cpu().numpy(), label=y_test.cpu().numpy().flatten())

# Train XGBoost Model
params = {"objective": "reg:squarederror", "eval_metric": "rmse"}
xgb_model = xgb.train(params, dtrain, num_boost_round=100)

# Predict
y_pred_xgb = xgb_model.predict(dtest)

# Compute Spearman correlation
spearman_corr_xgb, _ = spearmanr(y_test.cpu().numpy().flatten(), y_pred_xgb)
print(f"XGBoost Spearman Correlation: {spearman_corr_xgb:.4f}")


XGBoost Spearman Correlation: 0.4481


## LightGBM

In [48]:
import lightgbm as lgb

# Train LightGBM Model
lgb_train = lgb.Dataset(X_train.cpu().numpy(), label=y_train.cpu().numpy().flatten())

params = {"objective": "regression", "metric": "rmse"}
lgb_model = lgb.train(params, lgb_train, num_boost_round=100)

# Predict
y_pred_lgb = lgb_model.predict(X_test.cpu().numpy())

# Compute Spearman correlation
spearman_corr_lgb, _ = spearmanr(y_test.cpu().numpy().flatten(), y_pred_lgb)
print(f"LightGBM Spearman Correlation: {spearman_corr_lgb:.4f}")


LightGBM Spearman Correlation: 0.4600


# Ensemble of LightGBM & Linear Regression 
We use linear regression to weigh the different models in the ensemble instead of taking the mean. We use boostrapping to train each model on a different subset of the data.

In [101]:
# Step 1 - Train Ensemble
import lightgbm as lgb
import numpy as np
from scipy.stats import spearmanr
from tqdm import tqdm
from sklearn.utils import resample

# Define hyperparameters for LightGBM
# lgb_params = {"objective": "regression", "metric": "rmse",'learning_rate': 0.04931791757724383, 'num_leaves': 85, 'min_child_samples': 48, 'max_depth': 7, 'subsample': 0.5749253739933733, 'colsample_bytree': 0.8371746709650121,'verbosity':-1}
lgb_params = {'learning_rate': 0.07265158061393213, 'num_leaves': 99, 'min_child_samples': 40, 'max_depth': 5, 'subsample': 0.6919800894003186, 'colsample_bytree': 0.5493776294572092}

# Number of bootstrapped models
num_bootstraps = 100
lgb_boot_models = []
lgb_predictions = []


X_BOOTSTRAP_PROP = 20
print(f"Each bootstrap training size: {len(X_train) // num_bootstraps * X_BOOTSTRAP_PROP}")

progress = tqdm(total=num_bootstraps, desc="Training Bootstrapped LightGBM Models")

for i in range(num_bootstraps):
    # Bootstrap resampling
    X_resampled, y_resampled = resample(X_train.cpu().numpy(), y_train.cpu().numpy().flatten(), replace=True, n_samples=X_train.shape[0], random_state=i)

    # Train LightGBM model on resampled data
    lgb_train = lgb.Dataset(X_resampled, label=y_resampled)
    lgb_model = lgb.train(lgb_params, lgb_train, num_boost_round=100)
    
    # Store trained model
    lgb_boot_models.append(lgb_model)

    # Predict on test set
    y_pred_lgb = lgb_model.predict(X_test.cpu().numpy())
    lgb_predictions.append(y_pred_lgb)

    progress.update(1)

progress.close()

# Average predictions from bootstrapped models
y_pred_ensemble = np.mean(np.column_stack(lgb_predictions), axis=1)

# Compute Spearman correlation
spearman_corr_ensemble, _ = spearmanr(y_test.cpu().numpy().flatten(), y_pred_ensemble)

print(f"Bootstrapped LightGBM Ensemble Spearman Correlation: {spearman_corr_ensemble:.4f}")


Each bootstrap training size: 180


Training Bootstrapped LightGBM Models: 100%|██████████| 100/100 [01:20<00:00,  1.23it/s]


Bootstrapped LightGBM Ensemble Spearman Correlation: 0.4947


In [108]:
# Step 2: Train Meta Predictor

from sklearn.linear_model import Ridge

progress = tqdm(total=len(lgb_boot_models), desc="Generating `X_meta_train`")
X_meta_train = []
for m in lgb_boot_models:
    X_meta_train.append(m.predict(X_train.cpu().numpy()))
    progress.update(1)

progress.close()

X_meta_train = np.column_stack(X_meta_train)
y_meta_train = y_train.cpu().numpy().flatten()


# Train a Ridge Regression as the meta-model
meta_model = Ridge(alpha=1.0)
meta_model.fit(X_meta_train, y_meta_train)

## EVAL::
# Predict with stacked model
progress = tqdm(total=len(lgb_boot_models), desc="Predicting for `X_meta_test`")
X_meta_test = []
for m in lgb_boot_models:
    X_meta_test.append(m.predict(X_test.cpu().numpy()))
    progress.update(1)
progress.close()
X_meta_test = np.column_stack(X_meta_test)
y_meta_test = y_train.cpu().numpy().flatten()

y_meta_pred = meta_model.predict(X_meta_train)

# Compute Spearman correlation
print(y_meta_test.shape, y_meta_pred.shape)
spearman_corr_stacked, _ = spearmanr(y_meta_test, y_meta_pred)
print(f"Meta LightGBM Model Spearman Correlation: {spearman_corr_stacked:.4f}")


Predicting for `X_meta_test`: 100%|██████████| 100/100 [00:00<00:00, 561.63it/s]

(912,) (912,)
Meta LightGBM Model Spearman Correlation: 0.9146


# Predictions

In [103]:
df_test = pd.read_parquet('protein_embeddings_test.parquet')
X_unlabeled = df_test['Embedding'].values

In [104]:
X_unlabeled = np.vstack(X_unlabeled)
X_unlabeled = torch.tensor(X_unlabeled, dtype=torch.float32, device=device)

In [105]:
import numpy as np

# Compute UCB for LightGBM ensemble
# beta = 1.5  # Adjust exploration factor

y_pred_all = np.column_stack([model.predict(X_unlabeled.cpu().numpy()) for model in lgb_boot_models])
std_pred = np.std(y_pred_all, axis=1)    # Uncertainty (std dev)
mean_pred = meta_model.predict(y_pred_all)

In [106]:
# Rohan's Version
ucb_scores = mean_pred + 4.6399 * std_pred
top_mutations = np.argsort(ucb_scores)[-100:]
df_test.iloc[top_mutations]['mutant'].values

array(['E600P', 'A510Y', 'E274I', 'E168V', 'S392Y', 'S517Y', 'E3V',
       'D637R', 'A72Y', 'S493Y', 'E398V', 'Q554Y', 'D99L', 'E423I',
       'E340R', 'D80R', 'E428K', 'S492Y', 'S439Y', 'E468R', 'D360R',
       'E3Y', 'E379Y', 'E15W', 'K355Y', 'D401I', 'D381I', 'N486I', 'D27I',
       'E168R', 'N443Y', 'N486W', 'S205Y', 'H353I', 'S271Y', 'N648Y',
       'N486L', 'E299I', 'E3R', 'Q146Y', 'E600R', 'E359I', 'E340V',
       'D37I', 'D637F', 'S263Y', 'K26F', 'E278V', 'N648I', 'E600I',
       'Q476Y', 'D37R', 'E3W', 'C129Y', 'E640V', 'D187I', 'W420P',
       'N643Y', 'Q526Y', 'E415V', 'E15Y', 'E600Y', 'E135R', 'S9Y',
       'N488M', 'E274R', 'E600V', 'S483Y', 'Q570Y', 'D190I', 'S351Y',
       'P257I', 'Q163Y', 'S20Y', 'E640W', 'S411Y', 'Q484Y', 'E168I',
       'Q508Y', 'N546W', 'Q132Y', 'P342I', 'S409Y', 'W560I', 'E428R',
       'Q277Y', 'S113Y', 'N74Y', 'S429Y', 'S54Y', 'Q378Y', 'S183Y',
       'N614Y', 'E165V', 'D103I', 'N546Y', 'N486M', 'N546M', 'E358V',
       'N488Y'], dtype=object)

In [112]:
df_test["ucb_scores"] = ucb_scores
df_test

,mutant,Sequence,Embedding,ucb_scores
0,V1D,MDNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[0.1987293, 0.17874913, 0.011203687, 0.3522696...",0.715398
1,V1Y,MYNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[0.20150843, 0.16766307, 0.010554179, 0.351026...",0.747625
2,V1C,MCNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[0.19926624, 0.17924884, 0.0091279205, 0.35230...",0.720665
3,V1A,MANEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[0.19462435, 0.18044756, 0.006854555, 0.353092...",0.727539
4,V1E,MENEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[0.19793293, 0.18368268, 0.0076862425, 0.35060...",0.714736
...,...,...,...,...
11319,P655S,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[0.19556434, 0.1741927, 0.015259004, 0.3553321...",0.719264
11320,P655T,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[0.18834189, 0.17466125, 0.0054028304, 0.35398...",0.751059
11321,P655V,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[0.18966806, 0.18034382, -0.005223298, 0.35101...",0.751438
11322,P655A,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,"[0.18700847, 0.18019429, -0.0006385201, 0.3556...",0.747327


In [18]:
# Sai's Version
ucb_scores = std_pred
top_mutations = np.argsort(ucb_scores)[-100:]
df_test.iloc[top_mutations]['mutant'].values

array(['L589G', 'E231N', 'E468G', 'W499N', 'L631P', 'L155N', 'F579T',
       'L125G', 'L75T', 'L642G', 'A510G', 'F59G', 'A395G', 'I478G',
       'F514G', 'L639P', 'D624Y', 'E184N', 'A301G', 'A310G', 'L525T',
       'F64G', 'F206G', 'R265P', 'A83G', 'L471T', 'L404P', 'V282G',
       'L466A', 'L627T', 'R265N', 'L273N', 'I487G', 'E427N', 'L627G',
       'E274N', 'F230G', 'R387G', 'L155T', 'L10G', 'A327G', 'I364G',
       'D613N', 'D494N', 'L213N', 'F365G', 'E278G', 'L317M', 'I547G',
       'L407G', 'L125T', 'D288N', 'E414N', 'D244N', 'F563G', 'P626G',
       'L452G', 'L14G', 'L631T', 'F63C', 'I607G', 'L459P', 'L539P',
       'E640N', 'W560T', 'L525P', 'W348P', 'L75P', 'W454N', 'L147G',
       'L466T', 'E166N', 'D37Q', 'F431G', 'E46N', 'A349G', 'F256G',
       'I481G', 'A367G', 'V634G', 'L641G', 'L346T', 'M580G', 'I496G',
       'L300M', 'L155G', 'I181G', 'F405G', 'F238G', 'F434G', 'L620M',
       'D187G', 'I374G', 'L121G', 'L267G', 'W94P', 'F171G', 'L346M',
       'D91G', 'L639G'], dtype=

In [107]:
arr = list(df_test.iloc[top_mutations]['mutant'].values)
f = open('query.txt','w')
for i in arr:
    f.write(i+'\n')
f.close()

In [100]:
# TODO: Delete me
# test
sai_test = []
with open("query_backup.txt", "r") as f:
    data = f.read()
    sai_test = data.splitlines()
rohan_test = []
with open("query.txt", "r") as f:
    data = f.read()
    rohan_test = data.splitlines()

val = len(set(rohan_test) & set(sai_test))
print(val)

3


# Generating protein_embeddings_test.parquet

In [92]:
import torch
import esm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained ESM model and move it to GPU
esm_model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()  # Example: ESM-2 model
batch_converter = alphabet.get_batch_converter()
esm_model = esm_model.to(device)
esm_model.eval()  # Set to eval mode

# Extract ESM embeddings using GPU
def extract_esm_embedding(sequence):
    batch_labels, batch_strs, batch_tokens = batch_converter([(None, sequence)])
    batch_tokens = batch_tokens.to(device)  # Move input to GPU

    with torch.no_grad():
        results = esm_model(batch_tokens, repr_layers=[33], return_contacts=False)
    
    token_representations = results["representations"][33]  # Use final layer
    sequence_embedding = token_representations.mean(dim=1).squeeze().cpu().numpy()  # Move back to CPU for NumPy
    return sequence_embedding

Using device: cuda


In [24]:
df_test = pd.read_csv('test.csv')
df_test['Sequence'] = df_test['mutant'].apply(lambda x: seq[:int(x[1:-1])] + x[-1] + seq[int(x[1:-1])+1:])
df_test['Embedding'] = df_test['Sequence'].apply(lambda seq: extract_esm_embedding(seq))
df_test.to_parquet("protein_embeddings_test.parquet", engine="pyarrow")

KeyboardInterrupt: 

# Finding Optimal Hyperparams for LightGBM

In [93]:
#Best LightGBM Parameters: {'learning_rate': 0.04931791757724383, 'num_leaves': 85, 'min_child_samples': 48, 'max_depth': 7, 'subsample': 0.5749253739933733, 'colsample_bytree': 0.8371746709650121}
import optuna

def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 10, 100),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0),
    }

    lgb_train = lgb.Dataset(X_train.cpu().numpy(), label=y_train.cpu().numpy().flatten())
    lgb_model = lgb.train(params, lgb_train, num_boost_round=100)
    
    y_pred = lgb_model.predict(X_test.cpu().numpy())
    spearman_corr, _ = spearmanr(y_test.cpu().numpy().flatten(), y_pred)
    
    return spearman_corr

# Run Bayesian Optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# Best parameters
print("Best LightGBM Parameters:", study.best_params)


[I 2025-03-26 16:54:57,473] A new study created in memory with name: no-name-1438afd7-4cab-4c82-ba6d-03feabcc94d8
/tmp/ipykernel_1554313/1352214443.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
/tmp/ipykernel_1554313/1352214443.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
/tmp/ipykernel_1554313/1352214443.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "colsample_bytree": trial.suggest_uniform("colsam

Best LightGBM Parameters: {'learning_rate': 0.07265158061393213, 'num_leaves': 99, 'min_child_samples': 40, 'max_depth': 5, 'subsample': 0.6919800894003186, 'colsample_bytree': 0.5493776294572092}


# Best Beta
There is not much difference amongst the top options, so we just use 1.5 for simplicity.

In [97]:
import optuna
from scipy.stats import spearmanr

# Function to optimize beta
def optimize_beta(trial):
    beta = trial.suggest_uniform("beta", 0.1, 5.0)  # Search range
    
    ensemble_pred = np.column_stack([model.predict(X_test.cpu().numpy()) for model in lgb_boot_models])
    std_pred = np.std(ensemble_pred, axis=1)    # Uncertainty (std dev)
    mean_pred = meta_model.predict(ensemble_pred)

    ucb_scores = mean_pred + beta * std_pred


    # Compute Spearman correlation on these selected samples
    spearman_corr, _ = spearmanr(y_test.cpu().numpy().flatten(), ucb_scores)

    return spearman_corr

# Run optimization
study = optuna.create_study(direction="maximize")
study.optimize(optimize_beta, n_trials=50)

# Get best beta value
best_beta = study.best_params["beta"]
print(f"Optimized Beta: {best_beta:.4f}")

[I 2025-03-26 17:17:37,534] A new study created in memory with name: no-name-fb68060a-1bed-4d41-a5ee-4e270b054f99
/tmp/ipykernel_1554313/1317211863.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  beta = trial.suggest_uniform("beta", 0.1, 5.0)  # Search range
[I 2025-03-26 17:17:37,679] Trial 0 finished with value: 0.5060436079354995 and parameters: {'beta': 2.436792726322021}. Best is trial 0 with value: 0.5060436079354995.
/tmp/ipykernel_1554313/1317211863.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  beta = trial.suggest_uniform("beta", 0.1, 5.0)  # Search range
[I 2025-03-26 17:17:37,786] Trial 1 finished with value: 0.5058811052113006 and parameters: {'beta': 2.9506393718441983}. Best is tri

Optimized Beta: 4.6399


In [36]:
y_pred_stacked.shape

(228,)

# Retrain with Updated Information